In [27]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_transforms = transforms.Compose([
    # transforms.Resize(size=(32, 32)),
    transforms.ToTensor()
])
train_dataset = datasets.MNIST(root = "../../data/mnist", train = True, transform=train_transforms, download = True)
test_dataset = datasets.MNIST(root = "../../data/mnist", train = False, transform=train_transforms, download = True)

In [29]:
class NN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(in_channels, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [30]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out


In [31]:
# Hyperparameters
EPOCHS = 3
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.005
batch_size = 64

In [32]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [33]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()


In [34]:
for epoch in range(EPOCHS):
    for batch, (data, label) in enumerate(train_dataloader):
        data = data.to(device).squeeze(1)
        label = label.to(device)
        
        y_pred = model(data)
        loss = loss_fn(y_pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        


In [35]:
def check_accuracy(loader, model):


    num_correct = 0
    num_samples = 0
    model.eval()


    with torch.no_grad():
        for x, y in loader:

            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_dataloader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_dataloader, model)*100:.2f}")

Accuracy on training set: 9.03
Accuracy on test set: 8.92
